In [1]:
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, lower, length, regexp_extract

# Создание сессии Spark
spark = SparkSession.builder.appName("WikiAnalysis").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/02 19:06:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [48]:
df = spark.read.option("delimiter", "\t").csv('wiki.txt', schema="url STRING, title STRING, text STRING")
words_df = df.select(explode(split(col("text"), r'\W+')).alias("word"))
words_df = words_df.filter(col("word") != "").filter(~col("word").rlike(r'\d'))

### 1. Самое длинное слово

In [49]:
words_df.withColumn("length", length(col("word"))).orderBy(col("length").desc()).first()

Row(word='dreihundertvierundsechzigtausendachthundertneunzehn', length=51)

интересный факт который я узнал благодоря этому заданию, это слово переводится как "триста шестьдесят четыре тысячи восемьсот девятнадцать". дело в том, что числительные до миллиона в немецком пишутся слитно. 

### 2. Средняя длина слов


In [50]:
words_df.withColumn("length", length(col("word"))).agg({"length": "avg"}).collect()[0][0]

4.374960689351531

### 3. Самое частоупотребляемое слово (латинские буквы)

In [51]:
latin_words_df = words_df.filter(col("word").rlike(r'^[a-zA-Z]+$'))
latin_words_df.groupBy("word").count().orderBy(col("count").desc()).first()

Row(word='I', count=4259)

### 4. Слова, которые в половине случаев начинаются с большой буквы и встречаются больше 10 раз


In [52]:
from pyspark.sql.types import IntegerType

capitalized_words_df = df.select(explode(split(col("text"), r'\W+')).alias("word"))
capitalized_words_df = capitalized_words_df.withColumn("is_capitalized", col("word").rlike(r'^[A-ZА-Я].*').astype(IntegerType()))
capitalized_words_stats = capitalized_words_df.groupBy("word")\
    .agg({"is_capitalized": "avg", "*": "count"})\
    .where((col("avg(is_capitalized)") > 0.5) & (col("count(1)") > 10))
capitalized_words = capitalized_words_stats.select("word").collect()
[row["word"] for row in capitalized_words]


['XVII',
 'K',
 'Heaven',
 'Petroleum',
 'Warcraft',
 'Neue',
 'Sabbath',
 'Alt',
 'DNS',
 'Stars',
 'Assembler',
 'Total',
 'Russia',
 'VII',
 'LTE',
 'H2O',
 'Services',
 'Cr',
 'Outlook',
 'Slot',
 'PC',
 'Cl2',
 'PGP',
 'Telecom',
 'International',
 'XXVII',
 'Te',
 'API',
 'If',
 'Jabber',
 'One',
 'Water',
 'Capital',
 'Engine',
 'Application',
 'World',
 'Real',
 'NSFNet',
 'Technologies',
 'Al2O3',
 'ARM',
 'Nokia',
 'Ca',
 'Terra',
 'Sky',
 'Library',
 'Principia',
 'REXX',
 'Data',
 'Kit',
 'XVI',
 'Ernst',
 'Le',
 'Bank',
 'Usenet',
 'SO42',
 'XMPP',
 'Peugeot',
 'High',
 'ISO',
 'James',
 'KOI8',
 'MHz',
 'National',
 'Salt',
 'FTN',
 'POSIX',
 'Semiconductor',
 'Magic',
 'PS',
 'Science',
 'Delta',
 'AM',
 'IPv6',
 'Source',
 'PL',
 'Great',
 'NH2',
 'CaCO3',
 'AA',
 'MSN',
 'FAT32',
 'Road',
 'TGV',
 'Home',
 'PO4',
 'Little',
 'MX',
 'Review',
 'European',
 'Fi',
 'SOHO',
 'MThd',
 'Bell',
 'Motor',
 'RSS',
 'Fe',
 'Ogg',
 'Films',
 'F',
 'OpenOffice',
 'Subway',
 'On',


### 5. Устойчивые сокращения вида пр., др.

[]

### 6. Устойчивые сокращения вида т.п., н.э.

In [54]:
names_df = words_df.filter(col("word").rlike(r'\b[\wа-яА-Я]\.[\wа-яА-Я]\.\b'))
names_stats = names_df.groupBy("word").count().filter(col("count") > 1).collect()
[row["word"] for row in names_stats]

[]

### 7. Имена, употребляющиеся в статьях

In [53]:
names_df = words_df.filter(col("word").rlike(r'^[A-ZА-Я][a-zа-я]+$'))
names_stats = names_df.filter(length(col('word')) > 2).groupBy("word").count().filter(col("count") > 9).collect()
[row["word"] for row in names_stats]

['Heaven',
 'Petroleum',
 'Warcraft',
 'Neue',
 'Sabbath',
 'Alt',
 'Stars',
 'Assembler',
 'Total',
 'Russia',
 'Services',
 'Outlook',
 'Slot',
 'Telecom',
 'International',
 'Jabber',
 'One',
 'Water',
 'Capital',
 'Engine',
 'Application',
 'Days',
 'World',
 'Real',
 'Technologies',
 'Nokia',
 'Steve',
 'Terra',
 'Sky',
 'Aqua',
 'Library',
 'Principia',
 'Data',
 'Kit',
 'Ernst',
 'Bank',
 'Blu',
 'Usenet',
 'Peugeot',
 'High',
 'James',
 'National',
 'Salt',
 'Semiconductor',
 'Magic',
 'Science',
 'Delta',
 'Source',
 'Great',
 'Zend',
 'Road',
 'Home',
 'Little',
 'Review',
 'European',
 'Bell',
 'Motor',
 'Compaq',
 'Ogg',
 'Films',
 'Subway',
 'Adobe',
 'Los',
 'Europe',
 'Ray',
 'Floyd',
 'Records',
 'Nacional',
 'Rail',
 'Cyrix',
 'Sony',
 'Information',
 'Art',
 'Metal',
 'Association',
 'Common',
 'Networks',
 'Orange',
 'Commodore',
 'Management',
 'Box',
 'Public',
 'France',
 'Security',
 'Alcatel',
 'Motorola',
 'Oberverwaltungsgericht',
 'Energy',
 'Rock',
 'Award',